In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder,QuantileTransformer
import warnings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import scipy
from sklearn import preprocessing
from sklearn.impute import KNNImputer

In [ ]:
tr_df=pd.read_csv('transactions.csv')
ss=pd.read_csv('sample_submit.csv')
crm_goose=pd.read_csv('crm_goose.csv')
crm_data=pd.read_csv('crm_data.csv')

In [ ]:
crm_data=crm_data.merge(crm_goose, left_on='clnt_id', right_on='clnt_id')

In [ ]:
crm_data

In [ ]:
crm_data=crm_data.sort_values(by=['clnt_id'])
is_premium=crm_data['is_premium']

In [ ]:
col=crm_data.iloc[:,1:].columns
high_nans_list=[]
noncat_list=[]
for c in col:
    if crm_data[c].isna().sum()/ crm_data[c].shape>=0.2:
        high_nans_list.append(c)
    if len(crm_data[c].unique())>300:
        noncat_list.append(c)
noncat_list=list(set(noncat_list)-set(high_nans_list))        
crm_data.drop(high_nans_list,axis=1,inplace=True)
col=crm_data.iloc[:,1:].columns
nans_list=[]
for c in col:
    if crm_data[c].isna().sum()>0:
        print(c,crm_data[c].isna().sum(),'!!!!!!!!',len(crm_data[c].unique()))
        nans_list.append(c)

def condition(value):
    if 1000 <= value:
        return 2020-value
    else:
        return value
    
crm_data['clnt_age'] = crm_data['clnt_age'].apply(condition)

In [ ]:
trace1 = go.Bar(
    y=[len(crm_data[c].unique()) for c in crm_data.iloc[:,1:].columns],
    x=crm_data.iloc[:,1:].columns,
    name='Unique',
    marker=dict(
        color='blue'
    )
)
fig = go.Figure(data=trace1)
py.iplot(fig, filename='barchart')

In [ ]:
for col_number in [385,262,168,160,149,113,106,77,61,60,47,33,32,30,26,10,5,3,2]:
    col=crm_data.iloc[:,1:].columns
    for column in noncat_list:
        low_categorical_list=[]
        for c in col:
            if len(crm_data.iloc[:,1:][c].unique())<=col_number:
                if not crm_data.iloc[:,1:][c].isna().sum()>0:
                    low_categorical_list.append(c)
        if column in low_categorical_list:
            low_categorical_list.remove(column)
        print('LL',len(low_categorical_list))
        print(low_categorical_list)    
        crm_data[column]=crm_data[column].fillna(crm_data.groupby(low_categorical_list)[column].transform('mean'))
        print('NA',column,'***',crm_data[column].isna().sum())
        print('\n\n')

In [ ]:
for col_number in [385,262,168,160,149,113,106,77,61,60,47,33,32,30,26,10,5,3,2]:
    print("CN",col_number)
    col=crm_data.iloc[:,1:].columns
    for column in nans_list:
        low_categorical_list=[]
        for c in col:
            if len(crm_data.iloc[:,1:][c].unique())<=col_number:
                if not crm_data.iloc[:,1:][c].isna().sum()>0:
                    low_categorical_list.append(c)
        if column in low_categorical_list:
            low_categorical_list.remove(column)
            print('remove',column)
        print('LL',len(low_categorical_list))
        print(low_categorical_list)
        crm_data[column]=crm_data[column].fillna(crm_data.groupby(low_categorical_list)[column].transform('first'))
        print('NA',column,'***',crm_data[column].isna().sum())
        print('\n\n')
        

In [ ]:
crm_data

In [ ]:
le = preprocessing.LabelEncoder()
crm_data['clnt_device_get'] = le.fit_transform(crm_data['clnt_device_get'])
crm_data['clnt_device'] = le.fit_transform(crm_data['clnt_device'])
crm_data['is_premium'] = is_premium

In [ ]:
for i in [l for l in range(1,50)]:
    fig, ax = plt.subplots()
    sns.distplot(tr_df[tr_df['mcc_id']==i]['sum'],ax=ax)
    ax.set_title(i)

In [ ]:
trace1 = go.Bar(
    y=[tr_pivot[c].value_counts(normalize=True).iloc[0] for c in tr_pivot.iloc[:,1:-1].columns],
    x=tr_pivot.iloc[:,1:-1].columns,
    name='Unique',
    marker=dict(
        color='blue'
    )
)
fig = go.Figure(data=trace1)
py.iplot(fig, filename='barchart')

In [ ]:
crm_data

In [ ]:
stscaler = StandardScaler()
tr_mccid=tr_df.groupby(['clnt_id','mcc_id'])['sum'].agg([('negative' , lambda x : x[x < 0].sum()) , ('positive' , lambda x : x[x > 0].sum())]).reset_index()

In [ ]:
tr_mccid

In [ ]:
tr_mccid1=tr_mccid.groupby(['clnt_id'])['negative','positive'].sum().reset_index()
print(tr_mccid1)

In [ ]:
tr_pivot=tr_mccid.pivot(index=tr_mccid.index, columns='mcc_id')[['negative','positive']].reset_index()
tr_pivot.columns = ['_'.join(str(s).strip() for s in col if s) for col in tr_pivot.columns]
tr_pivot.reset_index()
tr_pivot=tr_pivot.fillna(0)
tr_pivot=tr_pivot.rename(columns={"index": "clnt_id"})
tr_mccid1=tr_mccid.groupby(['clnt_id'])['negative','positive'].sum().reset_index()
tr_pivot=tr_pivot.merge(tr_mccid1, left_on='clnt_id', right_on='clnt_id')
tr_pivot=tr_pivot.merge(crm_data, left_on='clnt_id', right_on='clnt_id')
bad_columns=[]
neg_columns=[f'negative_{i}' for i in range(1,51)]
pos_columns=[f'positive_{i}' for i in range(1,51)]


from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer()
tr_pivot[neg_columns]=scaler.fit_transform(tr_pivot[neg_columns])
tr_pivot[pos_columns]=scaler.fit_transform(tr_pivot[pos_columns])
tr_pivot.drop(['negative_31','negative_45','positive_2','positive_7','positive_10','positive_15','positive_18','positive_19','positive_23','positive_25',\
               'positive_28','positive_30','positive_31','positive_32','positive_34','positive_39','positive_45','positive_48','spend_points'],axis=1,inplace=True)  
stscaler = StandardScaler()
tr_pivot[['clnt_age','clnt_device','clnt_device_get','clnt_fortune','clnt_status','spend_level']] \
= scaler.fit_transform(tr_pivot[['clnt_age','clnt_device','clnt_device_get','clnt_fortune','clnt_status','spend_level']])
tr_pivot[['negative','positive','goose_agg_0',
       'goose_agg_1', 'goose_agg_2', 'goose_agg_3', 'goose_agg_4',
       'goose_agg_5', 'goose_agg_6', 'goose_agg_7', 'goose_agg_8',
       'goose_agg_10', 'goose_agg_11', 'goose_agg_12', 'goose_agg_13',
       'goose_agg_14', 'goose_agg_15', 'goose_agg_16', 'goose_agg_17',
       'goose_agg_18', 'goose_agg_19', 'goose_agg_20', 'goose_agg_21',
       'goose_agg_22', 'goose_agg_23', 'goose_agg_24', 'goose_agg_25',
       'goose_agg_26', 'goose_agg_27', 'goose_agg_28', 'goose_agg_29',
       'wild_goose_A_1', 'wild_goose_A_2', 'wild_goose_A_9','spend_last','clnt_income']]=scaler.fit_transform(tr_pivot[['negative','positive','goose_agg_0',
       'goose_agg_1', 'goose_agg_2', 'goose_agg_3', 'goose_agg_4',
       'goose_agg_5', 'goose_agg_6', 'goose_agg_7', 'goose_agg_8',
       'goose_agg_10', 'goose_agg_11', 'goose_agg_12', 'goose_agg_13',
       'goose_agg_14', 'goose_agg_15', 'goose_agg_16', 'goose_agg_17',
       'goose_agg_18', 'goose_agg_19', 'goose_agg_20', 'goose_agg_21',
       'goose_agg_22', 'goose_agg_23', 'goose_agg_24', 'goose_agg_25',
       'goose_agg_26', 'goose_agg_27', 'goose_agg_28', 'goose_agg_29',
       'wild_goose_A_1', 'wild_goose_A_2', 'wild_goose_A_9','spend_last','clnt_income']])

corr_matrix = tr_pivot.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
tr_pivot.drop(to_drop,axis=1,inplace=True)


for col in tr_pivot.iloc[:,1:-1].columns:
    if tr_pivot[col].value_counts(normalize=True).iloc[0]>0.95:
        bad_columns.append(col)
        
tr_pivot.drop(bad_columns,axis=1,inplace=True)  



In [ ]:
train_df =tr_pivot[tr_pivot['is_premium'].notna()]
test_df = tr_pivot[tr_pivot['is_premium'].isna()]

In [ ]:
train_df

In [ ]:
for col in train_df.columns[1:-1]:
    fig, axs = plt.subplots(ncols=2)
    sns.boxplot(x=train_df[col],ax=axs[0])
    sns.boxplot(x=test_df[col],ax=axs[1])
    axs[0].set_title(col)
    axs[1].set_title(col)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for col in train_df.columns[1:-1]:
    fig, axs = plt.subplots(ncols=2)
    sns.distplot(train_df[col],ax=axs[0])
    sns.distplot(test_df[col],ax=axs[1])
    axs[0].set_title(col)
    axs[1].set_title(col)

In [ ]:
train_df['is_premium'].value_counts()

In [ ]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor


In [ ]:
Y_train=train_df['is_premium']
X_train=train_df[las_columns]
X_test=test_df[las_columns]
Y_test=test_df['is_premium']
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.00001, random_state=0)
print(x_train.shape,x_valid.shape)

In [ ]:
x_train

In [ ]:
# Some useful parameters which will come in handy later on
ntrain = x_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 10 # set folds for out-of-fold prediction
kfold = StratifiedKFold(n_splits=5)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)[:,1] 
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(list(kfold.split(x_train,y_train))):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = (clf.predict_proba(x_test)>= 0.35).astype(bool)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
print(best_params_rf)
print(best_params_cb)
print(best_params_lgbm)
print(best_params_xgb)

In [ ]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=best_params_rf)
catboost = SklearnHelper(clf=CatBoostClassifier, seed=SEED, params=best_params_cb)
lgbm = SklearnHelper(clf=LGBMClassifier, seed=SEED, params=best_params_lgbm)
xgboost= SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, params=best_params_xgb)

In [ ]:
rf_oof_train, rf_oof_test = get_oof(rf, x_train, y_train, X_test) # 
cb_oof_train, cb_oof_test = get_oof(catboost,x_train, y_train, X_test) # 
lgbm_oof_train, lgbm_oof_test = get_oof(lgbm, x_train, y_train, X_test) # 
xgb_oof_train, xgb_oof_test = get_oof(xgboost, x_train, y_train, X_test)

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'CB': cb_oof_train.ravel(),
     'LGBM': lgbm_oof_train.ravel(),
     "XGB": xgb_oof_train.ravel()                          
    })
x_train = np.concatenate(( rf_oof_train, cb_oof_train, lgbm_oof_train,xgb_oof_train), axis=1)
x_test = np.concatenate(( rf_oof_test, cb_oof_test, lgbm_oof_test,xgb_oof_test), axis=1)

In [ ]:
NFOLDS = 5
rs=42
kf = StratifiedKFold(n_splits= NFOLDS)
params = {
        'max_depth': [4],
        'n_estimators': [1000],
        'min_child_weight': [1,2],
        'gamma': [2,5,10],
        'subsample': [0.8],
        'colsample_bytree': [1.0],
        'learning_rate': [0.01],
        'random_state':[rs],
        'objective': ['binary:logistic'],
        'nthread' : [-1],
        'scale_pos_weight': [1]
        }
xgboost = xgb.XGBClassifier()
gsXGB = GridSearchCV(xgboost, params, verbose=1, cv=kf.split(x_train,y_train), n_jobs=-1,scoring="f1_macro")
# Run the grid
gsXGB.fit(x_train,y_train)
XGB_best = gsXGB.best_estimator_
print('XGB',gsXGB.best_score_)
predictions = XGB_best.predict(x_test)

In [ ]:
ss=ss.sort_values(by=['clnt_id'])
subm=pd.DataFrame({'clnt_id':ss['clnt_id'],'is_premium':(XGB_best.predict_proba(x_test)[:,1] >= 0.35).astype(bool)})
subm=subm.astype({'is_premium': 'int64'})

In [ ]:
best['is_premium'].value_counts()

In [ ]:
print(accuracy_score(y_valid,predictions))

In [ ]:
random_state = 42
kfold = StratifiedKFold(n_splits=5)
classifiers = []
classifiers.append(LGBMClassifier(random_state=random_state))
classifiers.append(RandomForestClassifier(max_features=3,random_state=random_state))
classifiers.append(CatBoostClassifier(random_state = random_state))
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(SVC(random_state = random_state))
classifiers.append(xgb.XGBClassifier(random_state = random_state))
cv_results = []
for classifier in classifiers :
    print(classifier)
    cv_results.append(cross_val_score(classifier, x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

    
cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["LGBM","RandomForestClassifier","Catboost","Logreg","SVC","XGB"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

In [ ]:
best_params_rf=gsRFC.best_params_
best_params_cb=gsCB.best_params_
best_params_lgbm=gsLGBM.best_params_
best_params_xgb=gsXGB.best_params_
print(best_params_rf)
print(best_params_cb)
print(best_params_lgbm)
print(best_params_xgb)

In [ ]:
NFOLDS = 10
rs=42
kf = StratifiedKFold(n_splits= NFOLDS)
RFC = RandomForestClassifier()
rf_param_grid = {"max_depth": [None],"max_features": [3], "min_samples_split": [10], "min_samples_leaf": [ 3],
                "bootstrap": [False],
                "n_estimators" :[500],
                "criterion": ["gini"],
                'random_state':[rs]}
gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kf.split(x_train,y_train), scoring="accuracy", n_jobs= 4, verbose = 1)
gsRFC.fit(x_train,y_train)
RFC_best = gsRFC.best_estimator_
print("RF",gsRFC.best_score_)

clf = CatBoostClassifier()
params = {'iterations': [1000],
            'loss_function': ['CrossEntropy'],
         'random_state':[rs]}
gsCB = GridSearchCV(estimator=clf, param_grid=params, scoring="accuracy", cv=kf.split(x_train,y_train))
gsCB.fit(x_train,y_train)
CB_best = gsCB.best_estimator_
print('CB',gsCB.best_score_)

lgbm=LGBMClassifier()
gridParams = {
    'learning_rate': [0.01],
    'n_estimators': [500],
    'num_leaves': [12], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255], # large max_bin helps improve accuracy but might slow down training progress
    'colsample_bytree' : [0.64],
    'subsample' : [0.7],
    'reg_alpha' : [1.2],
    'reg_lambda' : [1.5],
    'random_state':[rs]
    }

gsLGBM = GridSearchCV(lgbm, gridParams, verbose=1, cv=kf.split(x_train,y_train), n_jobs=-1,scoring="accuracy")
# Run the grid
gsLGBM.fit(x_train,y_train)
LGBM_best = gsLGBM.best_estimator_
print('LGBM',gsLGBM.best_score_)
params = {
        'max_depth': [8,12],
        'n_estimators': [500,1000],
        'min_child_weight': [1],
        'gamma': [5],
        'subsample': [1.0],
        'colsample_bytree': [1.0],
        'learning_rate': [0.01],
        'random_state':[rs]
        }
xgboost = xgb.XGBClassifier()
gsXGB = GridSearchCV(xgboost, params, verbose=1, cv=kf.split(x_train,y_train), n_jobs=-1,scoring="accuracy")
# Run the grid
gsXGB.fit(x_train,y_train)
XGB_best = gsXGB.best_estimator_
print('XGB',gsXGB.best_score_)


In [ ]:
nrows = ncols = 2
fig, axes = plt.subplots(nrows = nrows, ncols = ncols, sharex="all", figsize=(15,15))

names_classifiers = [("RFC_best", RFC_best),("CB_best",CB_best),("LGBM_best",LGBM_best),("XGB_best",XGB_best)]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1][:40]
        g = sns.barplot(y=X_train.columns[indices][:40],x = classifier.feature_importances_[indices][:40] , orient='h',ax=axes[row][col])
        g.set_xlabel("Relative importance",fontsize=12)
        g.set_ylabel("Features",fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + " feature importance")
        nclassifier += 1

In [ ]:
indices = np.argsort(LGBM_best.feature_importances_)[::-1][:29]
las_columns=X_train.columns[indices][:29]

In [ ]:
las_columns

In [ ]:
final_columns=[spend_points,]

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best),('lgbm',LGBM_best),('cb',CB_best),("xgb",XGB_best)],
                           voting='soft', n_jobs=4)
votingC = votingC.fit(x_train, y_train)
predict=votingC.predict(x_valid)
print(accuracy_score(y_valid,predict))

In [ ]:
subm_rf['is_premium'].value_counts()

In [ ]:
subm_lgbm['is_premium'].value_counts()

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best),('cb',CB_best),('lgbm',LGBM_best),('xgb', XGB_best)],
                           voting='soft', n_jobs=4)
votingC = votingC.fit(x_train, y_train)

ss=ss.sort_values(by=['clnt_id'])
subm=pd.DataFrame({'clnt_id':ss['clnt_id'],'is_premium':(votingC.predict_proba(X_test)[:,1] >= 0.4).astype(bool)})
subm=subm.astype({'is_premium': 'int64'})

In [ ]:
subm.is_premium.value_counts()

In [ ]:
import gc

In [ ]:
gc.collect()